# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [62]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [63]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [64]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.admit,df.prestige,margins=True)

prestige,1.0,2.0,3.0,4.0,All
admit,,,,,
0,28,95,93,55,271
1,33,53,28,12,126
All,61,148,121,67,397


In [65]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.admit,df.prestige,rownames=['admit'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [66]:
x=np.array(df['prestige'])
unique, counts = np.unique(x,return_counts=True)
y=np.array(df['admit'])
unique, counts = np.unique(y,return_counts=True)
print np.asarray((unique,counts)).T

[[  0 271]
 [  1 126]]


#### 2.1 Create class or dummy variables for prestige 

In [67]:
dummies = pd.get_dummies(df['prestige'])
print(dummies)

     1.0  2.0  3.0  4.0
0      0    0    1    0
1      0    0    1    0
2      1    0    0    0
3      0    0    0    1
4      0    0    0    1
5      0    1    0    0
6      1    0    0    0
7      0    1    0    0
8      0    0    1    0
9      0    1    0    0
10     0    0    0    1
11     1    0    0    0
12     1    0    0    0
13     0    1    0    0
14     1    0    0    0
15     0    0    1    0
16     0    0    0    1
17     0    0    1    0
18     0    1    0    0
19     1    0    0    0
20     0    0    1    0
21     0    1    0    0
22     0    0    0    1
23     0    0    0    1
24     0    1    0    0
25     1    0    0    0
26     1    0    0    0
27     0    0    0    1
28     0    1    0    0
29     1    0    0    0
..   ...  ...  ...  ...
370    0    1    0    0
371    0    0    1    0
372    1    0    0    0
373    1    0    0    0
374    0    1    0    0
375    0    0    0    1
376    0    1    0    0
377    0    1    0    0
378    0    0    1    0
379    0    1   

In [68]:
dummy_ranks=pd.get_dummies(df['prestige'],prefix='prestige')
print dummy_ranks.head()

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1


#### 2.2 When modeling our class variables, how many do we need? 



#### Answer: 4, use 1 as baseline.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [69]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/Applications/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [70]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(df.admit,df.prestige,margins=True)

prestige  1.0  2.0  3.0  4.0  All
admit                            
0          28   95   93   55  271
1          33   53   28   12  126
All        61  148  121   67  397


In [71]:
print pd.crosstab(df['admit'],df['prestige'],rownames=['admit'],colnames=['prestige_1.0'])

prestige_1.0  1.0  2.0  3.0  4.0
admit                           
0              28   95   93   55
1              33   53   28   12


In [72]:
print pd.crosstab(handCalc.admit,handCalc['prestige_1.0'],margins=True)

prestige_1.0    0   1  All
admit                     
0             243  28  271
1              93  33  126
All           336  61  397


In [73]:
contingency_table = pd.crosstab(handCalc['admit'],handCalc['prestige_1.0'])
print contingency_table

prestige_1.0    0   1
admit                
0             243  28
1              93  33


In [74]:
print handCalc.describe()

            admit         gre         gpa  prestige_1.0  prestige_2.0  \
count  397.000000  397.000000  397.000000    397.000000    397.000000   
mean     0.317380  587.858942    3.392242      0.153652      0.372796   
std      0.466044  115.717787    0.380208      0.361070      0.484159   
min      0.000000  220.000000    2.260000      0.000000      0.000000   
25%      0.000000  520.000000    3.130000      0.000000      0.000000   
50%      0.000000  580.000000    3.400000      0.000000      0.000000   
75%      1.000000  660.000000    3.670000      0.000000      1.000000   
max      1.000000  800.000000    4.000000      1.000000      1.000000   

       prestige_3.0  prestige_4.0  
count    397.000000    397.000000  
mean       0.304786      0.168766  
std        0.460898      0.375017  
min        0.000000      0.000000  
25%        0.000000      0.000000  
50%        0.000000      0.000000  
75%        1.000000      0.000000  
max        1.000000      1.000000  


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [75]:
contingency_table.sum(axis=1)

admit
0    271
1    126
dtype: int64

In [76]:
contingency_table.sum(axis=0)

prestige_1.0
0    336
1     61
dtype: int64

In [77]:
P_prestige1=33/float(61)
print P_prestige1

0.540983606557


In [78]:
odds_prestige1=P_prestige1/(1-P_prestige1)
print odds_prestige1

1.17857142857


In [79]:
table = handCalc.groupby('prestige_1.0').sum().values
print(table)

[[  9.30000000e+01   1.96060000e+05   1.13608000e+03   1.48000000e+02
    1.21000000e+02   6.70000000e+01]
 [  3.30000000e+01   3.73200000e+04   2.10640000e+02   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [80]:
train_cols=handCalc.columns[1:]

In [81]:
logit=sm.Logit(handCalc['admit'],handCalc[train_cols])

In [82]:
result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [83]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        18:21:08   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_1.0    -3.8769      1.142     -3.39

In [84]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_1.0    0.020716
prestige_2.0    0.010494
prestige_3.0    0.005432
prestige_4.0    0.004382
dtype: float64


In [85]:
print result.conf_int()

                     0         1
gre           0.000074  0.004362
gpa           0.127619  1.431056
prestige_1.0 -6.116077 -1.637631
prestige_2.0 -6.739307 -2.374674
prestige_3.0 -7.472244 -2.958819
prestige_4.0 -7.664377 -3.196152


In [86]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns=['2.5%','97.5%','OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_1.0  0.002207  0.194440  0.020716
prestige_2.0  0.001183  0.093045  0.010494
prestige_3.0  0.000569  0.051880  0.005432
prestige_4.0  0.000469  0.040919  0.004382


In [87]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns=['2.5%','97.5%','OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_1.0  0.002207  0.194440  0.020716
prestige_2.0  0.001183  0.093045  0.010494
prestige_3.0  0.000569  0.051880  0.005432
prestige_4.0  0.000469  0.040919  0.004382


In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
lm = LogisticRegression()

In [79]:
lm.fit(handCalc[['prestige_1.0']], handCalc['admit'])
print lm.coef_
print lm.intercept_

[[ 1.02810535]]
[-0.93151263]


In [104]:
print np.exp(lm['prestige_1.0'])

TypeError: 'LogisticRegression' object has no attribute '__getitem__'

In [87]:
result = lm.fit(handCalc[['prestige_1.0']], handCalc['admit'])
print result

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [98]:
table = handCalc.groupby('prestige_1.0').sum().values
print(table)

[[  9.30000000e+01   1.96060000e+05   1.13608000e+03   1.48000000e+02
    1.21000000e+02   6.70000000e+01]
 [  3.30000000e+01   3.73200000e+04   2.10640000e+02   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [162]:
table = handCalc.groupby('admit').sum().values 
print(table)

[[  1.55440000e+05   9.07080000e+02   2.80000000e+01   9.50000000e+01
    9.30000000e+01   5.50000000e+01]
 [  7.79400000e+04   4.39640000e+02   3.30000000e+01   5.30000000e+01
    2.80000000e+01   1.20000000e+01]]


In [35]:
P_prestige234=93/float(336)
print P_prestige234

0.276785714286


In [36]:
odds_prestige234=P_prestige234/(1-P_prestige234)
print odds_prestige234

0.382716049383


#### 3.3 Calculate the odds ratio

In [39]:
OR=odds_prestige1/odds_prestige234
print OR

3.07949308756


In [44]:
OR=(P_prestige1/P_prestige234)
print OR

1.95452141724


In [88]:
N = np.array([[35,23], [20,29]])
LOR = np.log(N[0,0]) + np.log(N[1,1]) - np.log(np.log(N[0,1])) - np.log(N[1,0])

#### 3.4 Write this finding in a sentenance: 

Answer: 

#### 3.5 Print the cross tab for prestige_4

In [89]:
print pd.crosstab(handCalc.admit,handCalc['prestige_4.0'],margins=True)

prestige_4.0    0   1  All
admit                     
0             216  55  271
1             114  12  126
All           330  67  397


#### 3.6 Calculate the OR 

In [90]:
P_prestige4=12/float(67)
print P_prestige4
odds_prestige4=(P_prestige4/(1-P_prestige4))
print odds_prestige4
OR=odds_prestige4/(114/float(330))
print OR

0.179104477612
0.218181818182
0.631578947368


#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [91]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [92]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [93]:
train_cols=data.columns[1:]

#### 4.2 Fit the model

In [94]:
logit=sm.Logit(data['admit'],data[train_cols])
result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [95]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        18:25:26   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [96]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [97]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns=['2.5%','97.5%','OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [110]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [118]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [133]:
# recreate the dummy variables
combos.columns = ['gre','gpa','prestige_1.0','prestige_2.0','prestige_3.0','prestige_4.0','intercept']
dummy_ranks=pd.get_dummies(combos['prestige_1.0'], prefix='prestige_1.0')
dummy_ranks.columns=['prestige_1.0','prestige_2.0','prestige_3.0','prestige_4.0']

# keep only what we need for making predictions
cols_to_keep=['gre','gpa','prestige_2.0','intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:,'prestige':])

/Applications/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


ValueError: columns overlap but no suffix specified: Index([u'prestige_2.0'], dtype='object')

In [125]:
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [134]:
combos['admit_pred'] = result.predict(combos[train_cols])
print combos.head()

     gre       gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0  \
0  220.0  2.260000           1.0           1.0             0             0   
1  220.0  2.260000           2.0           1.0             1             0   
2  220.0  2.260000           3.0           1.0             0             1   
3  220.0  2.260000           4.0           1.0             0             0   
4  220.0  2.453333           1.0           1.0             0             0   

   intercept  admit_pred  
0          0    0.827672  
1          0    0.557380  
2          0    0.503951  
3          1    0.090492  
4          0    0.848114  


#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [135]:
def isolate_and_plot(variable):
    # isolate gre and class rank
    grouped = pd.pivot_table(combos, values=['admit_pred'], index=[variable, 'prestige'],
                            aggfunc=np.mean)

In [140]:
# make a plot
colors = 'rbgyrbgy'
for col in combos.prestige.unique():
    plt_data = grouped.ix[grouped.index.get_level_values(1)==col]
    pl.plot(plt_data.index.get_level_values(0), plt_data['admit_pred'],color=colors[int(col)])

    pl.xlabel(variable)
    pl.ylabel("P(admit=1)")
    pl.legend(['1', '2', '3', '4'], loc='upper left', title='Prestige')
    pl.title("Prob(admit=1) isolating " + variable + " and prestige")
    pl.show()

isolate_and_plot('gre')
isolate_and_plot('gpa')

AttributeError: 'DataFrame' object has no attribute 'prestige'